<a href="https://colab.research.google.com/github/dubovik-alexander/Netology-RSML-12/blob/main/HW3_RecSystems_HRS_ver_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание по теме «Гибридные рекомендательные системы»

### Задание

1. Датасет ml-latest.
2. Вспомнить подходы, которые мы разбирали.
3. Выбрать понравившийся подход к гибридным системам.
4. Написать свою.


**Задача:** \
Используя дата сет ml-latest (movielens). Рекомендовать пользователю топ 2 фильма на основе 3х жанров, в зависимости от дня недели (как правило в выходные просмотры могут быть семейные). Жанры определяем по его просмотрам, и проставленному рейтингу другими пользователями. Не предлагаем фильмы, которые пользователь уже смотрел.

#### Решение

In [2]:
! pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163757 sha256=b7d16c9f4d6fe0a242ae92e38bc1ae94b8a1c380c264b905cdcf604d5ee4590a
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [3]:
import os

import pandas as pd
import numpy as np
from datetime import datetime
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

from tqdm import notebook
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer


In [4]:
# # работает для windows, использую маленький размер данных, так как локальная машина не тенят большой объяем данных
# os.listdir('../ml_latest_small')
# prefix = '../ml_latest_small'

In [5]:
# # загружаем датафреймы
# links = pd.read_csv(os.path.join(prefix, 'links.csv'))
# movies = pd.read_csv(os.path.join(prefix, 'movies.csv'))
# ratings = pd.read_csv(os.path.join(prefix, 'ratings.csv'))
# tags = pd.read_csv(os.path.join(prefix, 'tags.csv'))

In [6]:
!wget https://github.com/ALKONDR/netology-recsys/archive/refs/heads/master.zip
!unzip master.zip

--2023-12-20 18:23:53--  https://github.com/ALKONDR/netology-recsys/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ALKONDR/netology-recsys/zip/refs/heads/master [following]
--2023-12-20 18:23:54--  https://codeload.github.com/ALKONDR/netology-recsys/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [  <=>               ]   1.06M  4.06MB/s    in 0.3s    

2023-12-20 18:23:54 (4.06 MB/s) - ‘master.zip’ saved [1111929]

Archive:  master.zip
dfe2a910caf170a1f0fd2174867169ce737c9dc7
   creating: netology-recsys-master/
   creating: netology-rec

In [84]:
links = pd.read_csv('netology-recsys-master/lecture-1/links.csv')
movies = pd.read_csv('netology-recsys-master/lecture-1/movies.csv')
ratings = pd.read_csv('netology-recsys-master/lecture-1/ratings.csv')
tags = pd.read_csv('netology-recsys-master/lecture-1/tags.csv')

In [85]:
movies.shape

(9742, 3)

##### Сначала подготовим данные (ratings)

In [86]:
# Так как для анализа буду использовать будни и выходные, то перевожу время в бинарный тип данных

# Преобразование временного штампа в объект Timestamp
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

# Добавление новой колонки, которая будет содержать 1 для рабочих дней и 0 для выходных
ratings['is_workday'] = np.where(ratings['timestamp'].dt.weekday < 5, 1, 0)

ratings['timestamp'] = ratings['timestamp'].apply(lambda x: int(x.timestamp()))

In [87]:
ratings.shape

(100836, 5)

In [88]:
tags.shape

(3683, 4)

##### Подготовим данные (genres)

In [89]:
# Приводим все значения в колонке 'tag' к нижнему регистру
tags['tag'] = tags['tag'].str.lower()

In [90]:
# объединяем теги по каждому фильму и делаем теги уникальными

tag_list = []
movie_list = []

for k, v in notebook.tqdm(tags.groupby('movieId').tag):
    tag_list.append(' '.join(v.values))
    movie_list.append(k)

  0%|          | 0/1572 [00:00<?, ?it/s]

In [91]:
# Создаем  датафрейм movieId - tag

movies_t = pd.DataFrame(
    {
        'movie_id': movie_list,
        'tag': tag_list
    }
)
movies_t.head(5)


,movie_id,tag
0,1,pixar pixar fun
1,2,fantasy magic board game robin williams game
2,3,moldy old
3,5,pregnancy remake
4,7,remake


In [92]:
# Далее убираем все лишние пробелы, дефисы и знак разделения у жанров
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [93]:
movies_g = movies.genres.apply(change_string).tolist()

In [94]:
# Создание объекта TfidfVectorizer
vectorizer_g = TfidfVectorizer()
vectorizer_t = TfidfVectorizer()

# Преобразование признаков genres и tags в пространство TF-IDF

movie_genres_g = vectorizer_g.fit_transform(movies_g)
movie_genres_t = vectorizer_t.fit_transform(movies_t['tag'].values)

In [95]:
# Преобразования разреженной матрицы в плотную матрицу (из лекции)
dense_matrix_g = movie_genres_g.todense()

# Преобразования плотной матрицы в датафрейм (из лекции)
df_g = pd.DataFrame(dense_matrix_g, index=movies.index)
df_t = pd.DataFrame(movie_genres_t.toarray())

# название колонок из фичей
df_g.columns = vectorizer_g.get_feature_names_out()
df_t.columns = vectorizer_t.get_feature_names_out()

df_t.insert(loc = 0, column ='movieId', value = movies_t.movie_id)
df_t.head(1)
df_t.shape


(1572, 1745)

In [96]:
df_g.insert(loc= 0, column='movieId', value=movies.movieId)
df_g.head(1)
df_g.shape

(9742, 21)

In [97]:
ratings.shape

(100836, 5)

##### Объединяем dataframe's с жанрами и тегами и названием фильма

In [98]:

df_final= ratings.merge(df_g, on='movieId')

df_final = df_final.merge(df_t, on='movieId')
df_final = df_final.merge(movies, on='movieId')
df_final.head()


,userId,movieId,rating,timestamp,is_workday,action_x,adventure_x,animation_x,children_x,comedy_x,...,younger,your,zellweger,zither,zoe,zombie,zombies,zooey,title,genres
0,1,1,4.0,964982703,0,0.0,0.416846,0.516225,0.504845,0.267586,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,1,0.0,0.416846,0.516225,0.504845,0.267586,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,1,0.0,0.416846,0.516225,0.504845,0.267586,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,1,0.0,0.416846,0.516225,0.504845,0.267586,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,1,0.0,0.416846,0.516225,0.504845,0.267586,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [99]:
gener_columns = df_final.columns[6:-2].tolist()

##### Рекомендация:

In [102]:
def user_films_genre(user_id, last_film_count = 20, current_day = 0, top_genres_count = 3):
    ''' Жанры, по умолчанию предлагаем три жанра, из самых последних фильмов пользователя с учетом выходных и рабочих дней,
        по умолчанию стоят выходные - current_day = 0'''
    # отбираем сначала все строки по текущему пользователю
    films_current_user = df_final[df_final['userId'] == user_id]

    # тригер (бинарный) выходной день просмотра - 0, будни - 1
    films_current_user = df_final[df_final['is_workday'] == current_day]

    # cортируем по времени просмотра, для определения какие жанры были последними
    films_current_user = films_current_user.sort_values('timestamp', ascending=False)
    last_films_list = films_current_user.head(last_film_count)

    # Подсчитаем фильмы для каждого жанра что бы определить какие жанры больше нравяться (смотрел пользователь).
    last_films_list = last_films_list[gener_columns].replace(0,np.nan).count(axis=0).reset_index()

    genres = last_films_list.sort_values(0, ascending=False).head(top_genres_count)['index'].values

    return list(genres)

In [103]:
genres

['drama_x', 'thriller_x', 'crime_x']

In [104]:
def get_expert_recommendations(user_id, genres, expert_value = 100, expert_film_list_count = 20):
    ''' For user_id ask expert who in TOP-100 of user_id who seen films of genres about list of recommendations not from seen by user_id '''

    # создаем ДФ все записи без юзера по которому делаем рекомендацию
    df_except_user = df_final[df_final['userId']!=user_id]

    # Подсчитаем фильмы для каждого пользователя
    df_except_user = df_except_user.replace(0,np.nan).groupby('userId').agg('count').reset_index()

    # ТОП экспертов
    experts = df_except_user.sort_values(genres, ascending=False).head(expert_value)['userId'].values

    # Найти фильмы этих жанров, просмотренные экспертами и не просмотренные пользователем.
    films_seen_expert = df_final[df_final['userId'] == user_id]['movieId'].unique()

    films_not_seen_expert = df_final.query("userId in @experts and movieId not in @films_seen_expert")

    # Фильтровать films_not_seen_expert только по жанрам:
        # 1: Уникальные идентификаторы фильмов из DataFrame 'films_not_seen_expert'.
    unique_movieIds = films_not_seen_expert['movieId'].unique()

        # 2: Отфильтруем df_final на основе уникальных идентификаторов MovieId.
    filtered_df = df_final[df_final['movieId'].isin(unique_movieIds)]

        # 3: Выберем столбцы «movieId» и «genres» из отфильтрованного DataFrame.
    selected_df = filtered_df[['movieId'] + genres]

        # 4: Заменим 0 на NaN и удалим строки со значениями NaN.
    cleaned_df = selected_df.replace(0, np.nan).dropna()

        # 5: Получим значения «movieId» в виде массива.
    movieId_same_genres = cleaned_df['movieId'].values

    films_not_seen_expert = films_not_seen_expert[films_not_seen_expert['movieId'].isin(movieId_same_genres)]

    # Find list of recommendations from experts (from films seen by experts but not seen by user_id)
    dataset = films_not_seen_expert[['userId', 'movieId', 'rating']]
    reader = Reader(rating_scale=(0.5, 5))
    data = Dataset.load_from_df(dataset, reader)
    trainset, _ = train_test_split(data, test_size=0.01)
    algorithm = SVD(n_epochs=15, verbose=False, biased=True, n_factors=15)
    algorithm.fit(trainset)
    recom = pd.DataFrame(movieId_same_genres, columns=['movieId'])
    recom['Score'] = recom.apply(lambda r: algorithm.predict(user_id, r['movieId']).est, axis=1)
    recom = recom.sort_values('Score', ascending=False).head(expert_film_list_count)

    return recom



In [106]:
# top_by_experts = get_expert_recommendations(user_id, genres)
# top_by_experts

In [107]:
def rating_for_user(user_id, top_by_experts, film_count = 5):
    ''' Get only (film_count) films with highest average rating '''
    movieIds = top_by_experts['movieId'].values
    df = df_final[df_final['movieId'].isin(movieIds)][['movieId', 'rating']]
    # Calc mean rating for recommended films by experts and get only top (film_count) films
    df = df.groupby('movieId').agg('mean').reset_index().sort_values('rating', ascending = False).head(film_count)
    # Add title to result
    df = df.merge(movies, on='movieId')[['movieId', 'title', 'rating']]

    return df

In [108]:
user_id = 2 # id пользователя для которого делаем предсказание
top_genres_count = 3
genres = user_films_genre(user_id)
top_by_experts = get_expert_recommendations(user_id, genres)
film_count = 3 # кол-во фильмов которые хотим предложить
current_day = 0 # маркер выходного дня, по умолчанию стоят выходные - current_day = 0, 1 будни (предполагаем что фильмы смотрят по одним id и взрослые и дети)
result = rating_for_user(user_id, top_by_experts, film_count)

In [109]:
result

,movieId,title,rating
0,122912,Avengers: Infinity War - Part I (2018),4.000000
1,924,2001: A Space Odyssey (1968),3.894495
